This Jupyter notebook contains a set of sample codes that are necessary for obtaining the calculation results in the paper titled "Functional Output Regression for Machine Learning in Materials Science". For readers who want to test the model without training the model from scratch, we provide three pre-trained models that are available for download: https://github.com/yoshida-lab/XenonPy/releases/download/v0.6.3/pretrained_models.zip

In [260]:
# python packages used in the sample codes

from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.utils.data
from torch import optim

from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem import Draw
from rdkit import rdBase, Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem.AtomPairs import Pairs, Torsions

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# user-friendly print out
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


### Preparation

#### Utility function: define RBFKernel class to be the radial basis function kernel that will be used in our models

In [234]:
import numpy as np

from typing import Union, Sequence, Callable, Tuple


class RBFKernel():
    def __init__(self, sigmas_squared: Union[float, int, np.ndarray, Sequence], hight=10, *, dtype='float32'):
        """
        Radial Basis Function (RBF) kernel function.
        Ref: https://en.wikipedia.org/wiki/Radial_basis_function_kernel

        Parameters
        ----------
        sigmas:
            The squared standard deviations (SD).
            Can be a single number or a 1d array-like object.
        x_i: np.ndarray
            Should be a 1d array.
        x_j: np.ndarray
            Should be a 1d array.

        Returns
        -------
        np.ndarray
            Distribution under RBF kernel.

        Raises
        ------
        ValueError
            Raise error if sigmas has wrong dimension.
        """
        sigmas_squared = np.asarray(sigmas_squared)
        if sigmas_squared.ndim == 0:
            sigmas_squared = sigmas_squared[np.newaxis]
        if sigmas_squared.ndim != 1:
            raise ValueError('parameter `sigmas_squared` must be a array-like object which has dimension 1')
        self._sigmas_squared = sigmas_squared
        self.hight = hight
        self.dtype = dtype
        
    @property
    def sigmas_squared(self):
        return np.copy(self._sigmas_squared)
    
    def __call__(self, x_i: np.ndarray, x_j: Union[np.ndarray, int, float]):
        # K(x_i, x_j) = exp(-||x_i - x_j||^2 / (2 * sigma^2))
        p1 = np.power(np.expand_dims(x_i, axis=x_i.ndim) - x_j, 2)
        p2 = self._sigmas_squared * 2
        dists = self.hight * np.exp(-np.expand_dims(p1, axis=p1.ndim) / p2)

        if dists.shape[2] == 1:
            return dists[:, :, 0].astype(self.dtype)
        return dists.astype(self.dtype)


#### Model definition: define KernelNet class to be the neural network based kernel function model

In [294]:
from xenonpy.model import SequentialLinear

# model parameter initialization
@torch.no_grad()
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm1d):
        nn.init.normal_(m.weight, 1.0, 0.02)
        nn.init.constant_(m.bias, 0)

        
class KernelNet(nn.Module):
    def __init__(self, *,
                 n_neurons=(1024, 896, 512, 256, 128),
                 activation_func=nn.LeakyReLU(0.2, inplace=True),
                 kernel_grids: Union[int, np.ndarray] = 128,
                 wavelength_points: Union[int, np.ndarray] =181,
                 kernel_func=RBFKernel(sigmas_squared=0.05, hight=1),
                ):
        super().__init__()

        # initializing fixed params (not updated through training)
        if isinstance(kernel_grids, int):
            kernel_grids = np.linspace(0, 1, kernel_grids)
        if isinstance(kernel_grids, np.ndarray):
            self.kernel_grids = kernel_grids
        else:
            raise ValueError('kernel_grids error!')
        
        if isinstance(wavelength_points, int):
            wavelength_points = np.linspace(0, 1, wavelength_points)
        if isinstance(wavelength_points, np.ndarray):
            self.wavelength_points = wavelength_points
        else:
            raise ValueError('wavelength_points error!')
        
        # fully connected layers
        self.fc_layers = SequentialLinear(
            in_features=n_neurons[0], out_features=n_neurons[-1], h_neurons=n_neurons[1:-1], h_activation_funcs=activation_func
        )
        
        # baseline parameters
        self.mu = torch.nn.Parameter(torch.zeros(wavelength_points.size))
        
        # kernel
        self.kernel = torch.from_numpy(
            kernel_func(self.kernel_grids, self.wavelength_points)
        )
        
    def to(self, *arg, **kwarg):
        self.kernel = self.kernel.to(*arg, **kwarg)
        return super().to(*arg, **kwarg)

    def forward(self, fingerprint_input):
        x = self.fc_layers(fingerprint_input)
        x = x.view(x.size(0), self.kernel_grids.size, -1) 
        x = x * self.kernel
        x = torch.sum(x, dim=1)
        x = self.mu + x
        return x
    

### Loading data

In [298]:
# load csv file
data_file = "data/Dataset_I.csv"
input_file = pd.read_csv(data_file,sep=",")

# Figure
out_result = Path("spectrum_results")
out_result.mkdir(exist_ok=True, parents=True)

# drop "OC(=O)C(=C\C1=CC=[Cl]C=[Cl]1)\C1=CN=CC=C1" because of Explicit valence error
input_file = input_file[input_file.SMILES != "OC(=O)C(=C\C1=CC=[Cl]C=[Cl]1)\C1=CN=CC=C1"]

# load smiles and spectrum data
file_smiles_spectrum = input_file.values
smiles_list = (file_smiles_spectrum[:,1])
spectrum_list = file_smiles_spectrum[:,2:]
spectrum_list = np.array(spectrum_list, dtype='float32')

print("number of data:", len(smiles_list))


number of data: 949


#### convert smiles to fingerprint

In [6]:
## convert smiles to mol file
mol_list = [Chem.MolFromSmiles(s) for s in smiles_list if s is not None]
    
## Morgan fingerprint (ECFP) with radius=3 and 1024 bits
fps_bit = [AllChem.GetMorganFingerprintAsBitVect(m, 3, 1024) for m in mol_list]
fps_list = []
for fps in fps_bit:
    fps_arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fps, fps_arr)
    fps_list.append(fps_arr)
fps_list = np.array(fps_list, dtype='float32')

fps_list_max = np.amax(fps_list)
fps_list_min = np.amin(fps_list)
fps_norm = ((fps_list - fps_list_min) / (fps_list_max - fps_list_min))
X_data = torch.from_numpy(fps_norm)


spectrum_list_max = np.amax(spectrum_list)
spectrum_list_min = np.amin(spectrum_list)
spectrum_norm = ((spectrum_list - spectrum_list_min) / (spectrum_list_max - spectrum_list_min))
Y_data = torch.from_numpy(spectrum_norm)
data_number = torch.zeros((X_data.shape[0],1))
Y_data = torch.cat((data_number, Y_data), dim=1)
for i in range(Y_data.shape[0]):
    number = torch.tensor([i]) 
    Y_data[i][0] = number
    
X_data.shape
Y_data.shape


torch.Size([949, 1024])

torch.Size([949, 182])

### Set up hyper-params

#### 1. model training parameters

In [270]:
n_epoch = 3000 # epoch
lr = 0.0002 # learning rate
ngpu = 2 # number of GPU
display_interval = 100

model_path = Path('model/dataset1')
model_path.mkdir(exist_ok=True, parents=True)

In [241]:
case = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

best_layer_number = 4 # number of layers
best_variance = 0.0005 # the variange of RBF kernel
best_length = 0.5 # the length of RBF kernel 

# split train:val:test
split_size = 0.316
split_val_size = 0.5


In [239]:
start_layer = best_layer_number
end_layer = start_layer + 1

hyper_id = [5, 6, 7, 8]
hyper_order = 3
start_hyper = hyper_id[hyper_order]
end_hyper = start_hyper + 1

#### 2. model parameters

In [243]:
wavelength_points = 181
kernel_grids = 128
n_neurons = [
    (1024, 512, 128),                 # 1 hidden layer
    (1024, 512, 256, 128),            # 2 hidden layer
    (1024, 896, 512, 256, 128),       # 3 hidden layer
    (1024, 896, 640, 512, 256, 128),  # 4 hidden layer
]
kernel_hypers = (
    [10, 0.00005], [5, 0.00005], [1, 0.00005], [0.5, 0.00005],
    [10, 0.0005], [5, 0.0005], [1, 0.0005], [0.5, 0.0005],
    [10, 0.00025], [5, 0.00025], [1, 0.00025], [0.5, 0.00025],
)

### Begin model training

In [233]:
# fixing random seeds for reproducing the results
np.random.seed(0)
torch.manual_seed(0)


#### train with different hyperparameters

In [296]:
# repeat for different number of layers in the neural network
for layer_number in range(start_layer, end_layer):
    # repeat for different sigma values for the kernel
    for hyper_number in range(start_hyper, end_hyper):
        # repeat for three times to check sensitivity to data splitting
        for random_number in range(3):
            # split data with different random seed
            X_train, X_test_val, Y_train, Y_test_val = train_test_split(X_data, Y_data, test_size=split_size, random_state=random_number) 
            X_test, X_val, Y_test, Y_val = train_test_split(X_test_val, Y_test_val, test_size=split_val_size, random_state=random_number)
            Dataset_train = torch.utils.data.TensorDataset(X_train, Y_train)
            loader_train = torch.utils.data.DataLoader(Dataset_train, batch_size=50, shuffle=True)
            Dataset_val = torch.utils.data.TensorDataset(X_val, Y_val)
            loader_val = torch.utils.data.DataLoader(Dataset_val, shuffle=False)

            # bulid model
            netG = KernelNet(
                n_neurons=n_neurons[layer_number - 1],
                wavelength_points=wavelength_points,
                kernel_grids=kernel_grids,
                kernel_func=RBFKernel(
                    sigmas_squared=kernel_hypers[hyper_number - 1][1],
                    hight=kernel_hypers[hyper_number - 1][0]
                ),
            ).to(device)

            # Handle multi-gpu if desired

            # if (device.type == 'cuda') and (torch.cuda.device_count() > ngpu):
            #     netG = nn.DataParallel(netG, device_ids=range(ngpu))            
            netG = netG.apply(weights_init)

            # setup optimizer for training model
            regu = torch.Tensor([1]).to(device) # parameter regularization
            criterion = nn.MSELoss()
            optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999), weight_decay=1e-5)
            
            # train with fixed number of epochs
            losses = [] 
            print(f"training {layer_number} hidden layer model with hyper_number {hyper_number} and random_number {random_number}...")
            for epoch in range(n_epoch):
                for itr, data in enumerate(loader_train):
                    real_fps = data[0][:,:].to(device)
                    real_spectrum = data[1][:,1:].to(device)
                    optimizerG.zero_grad()

                    pred_spectrum = netG(real_fps)

                    mu_parameter = netG.mu
                    mu_para_diff = torch.diff(mu_parameter)
                    delta_mu = torch.sum(torch.pow((mu_para_diff),2))
                    
                    loss = criterion(pred_spectrum, real_spectrum[:,:]) + regu * delta_mu

                    loss.backward()
                    optimizerG.step()
                    losses.append(loss.item())

                    if epoch % display_interval == 0 and itr % display_interval == 0:
                        print('[{}/{}][{}/{}] Loss: {:.7f} '.format(epoch + 1, n_epoch, itr + 1, len(loader_train), loss.item()))
                        
            # save model
            PATH_G = '{}/generator_time{:03d}_layer{:03d}_hyper{:03d}.pth'.format(model_path, random_number, layer_number, hyper_number) 
            torch.save(netG.state_dict(), PATH_G)


GNet(
  (fc_layers): SequentialLinear(
    (layer_0): LinearLayer(
      (linear): Linear(in_features=1024, out_features=896, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
      (normalizer): BatchNorm1d(896, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (layer_1): LinearLayer(
      (linear): Linear(in_features=896, out_features=640, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
      (normalizer): BatchNorm1d(640, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (layer_2): LinearLayer(
      (linear): Linear(in_features=640, out_features=512, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
      (normalizer): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (layer_3): 

training 4 hidden layer model with hyper_number 8 and random_number 0...
[1/3000][1/13] Loss: 0.9513877 
[101/3000][1/13] Loss: 0.0049158 
[201/3000][1/13] Loss: 0.0015354 
[301/3000][1/13] Loss: 0.0025721 
[401/3000][1/13] Loss: 0.0016615 
[501/3000][1/13] Loss: 0.0015076 
[601/3000][1/13] Loss: 0.0017643 
[701/3000][1/13] Loss: 0.0011007 
[801/3000][1/13] Loss: 0.0012107 
[901/3000][1/13] Loss: 0.0005568 
[1001/3000][1/13] Loss: 0.0009820 
[1101/3000][1/13] Loss: 0.0014051 
[1201/3000][1/13] Loss: 0.0009783 
[1301/3000][1/13] Loss: 0.0006637 
[1401/3000][1/13] Loss: 0.0008219 
[1501/3000][1/13] Loss: 0.0006080 
[1601/3000][1/13] Loss: 0.0007286 
[1701/3000][1/13] Loss: 0.0009694 
[1801/3000][1/13] Loss: 0.0008945 
[1901/3000][1/13] Loss: 0.0005525 
[2001/3000][1/13] Loss: 0.0005456 
[2101/3000][1/13] Loss: 0.0008501 
[2201/3000][1/13] Loss: 0.0004946 
[2301/3000][1/13] Loss: 0.0003780 
[2401/3000][1/13] Loss: 0.0005973 
[2501/3000][1/13] Loss: 0.0005007 
[2601/3000][1/13] Loss: 0.000

GNet(
  (fc_layers): SequentialLinear(
    (layer_0): LinearLayer(
      (linear): Linear(in_features=1024, out_features=896, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
      (normalizer): BatchNorm1d(896, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (layer_1): LinearLayer(
      (linear): Linear(in_features=896, out_features=640, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
      (normalizer): BatchNorm1d(640, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (layer_2): LinearLayer(
      (linear): Linear(in_features=640, out_features=512, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
      (normalizer): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (layer_3): 

training 4 hidden layer model with hyper_number 8 and random_number 1...
[1/3000][1/13] Loss: 0.8627182 
[101/3000][1/13] Loss: 0.0093594 
[201/3000][1/13] Loss: 0.0030646 
[301/3000][1/13] Loss: 0.0041787 
[401/3000][1/13] Loss: 0.0021683 
[501/3000][1/13] Loss: 0.0025061 
[601/3000][1/13] Loss: 0.0014158 
[701/3000][1/13] Loss: 0.0016073 
[801/3000][1/13] Loss: 0.0020572 
[901/3000][1/13] Loss: 0.0013698 
[1001/3000][1/13] Loss: 0.0013439 
[1101/3000][1/13] Loss: 0.0011626 
[1201/3000][1/13] Loss: 0.0013354 
[1301/3000][1/13] Loss: 0.0007292 
[1401/3000][1/13] Loss: 0.0008215 
[1501/3000][1/13] Loss: 0.0005767 
[1601/3000][1/13] Loss: 0.0007712 
[1701/3000][1/13] Loss: 0.0009511 
[1801/3000][1/13] Loss: 0.0006395 
[1901/3000][1/13] Loss: 0.0005922 
[2001/3000][1/13] Loss: 0.0004482 
[2101/3000][1/13] Loss: 0.0015007 
[2201/3000][1/13] Loss: 0.0005241 
[2301/3000][1/13] Loss: 0.0005661 
[2401/3000][1/13] Loss: 0.0005935 
[2501/3000][1/13] Loss: 0.0004629 
[2601/3000][1/13] Loss: 0.000

GNet(
  (fc_layers): SequentialLinear(
    (layer_0): LinearLayer(
      (linear): Linear(in_features=1024, out_features=896, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
      (normalizer): BatchNorm1d(896, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (layer_1): LinearLayer(
      (linear): Linear(in_features=896, out_features=640, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
      (normalizer): BatchNorm1d(640, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (layer_2): LinearLayer(
      (linear): Linear(in_features=640, out_features=512, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
      (normalizer): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (layer_3): 

training 4 hidden layer model with hyper_number 8 and random_number 2...
[1/3000][1/13] Loss: 1.0644132 
[101/3000][1/13] Loss: 0.0099005 
[201/3000][1/13] Loss: 0.0022077 
[301/3000][1/13] Loss: 0.0020309 
[401/3000][1/13] Loss: 0.0017221 
[501/3000][1/13] Loss: 0.0014918 
[601/3000][1/13] Loss: 0.0014006 
[701/3000][1/13] Loss: 0.0013380 
[801/3000][1/13] Loss: 0.0011670 
[901/3000][1/13] Loss: 0.0013053 
[1001/3000][1/13] Loss: 0.0006377 
[1101/3000][1/13] Loss: 0.0009901 
[1201/3000][1/13] Loss: 0.0008227 
[1301/3000][1/13] Loss: 0.0011035 
[1401/3000][1/13] Loss: 0.0006691 
[1501/3000][1/13] Loss: 0.0009261 
[1601/3000][1/13] Loss: 0.0007164 
[1701/3000][1/13] Loss: 0.0007345 
[1801/3000][1/13] Loss: 0.0006400 
[1901/3000][1/13] Loss: 0.0005412 
[2001/3000][1/13] Loss: 0.0003890 
[2101/3000][1/13] Loss: 0.0004704 
[2201/3000][1/13] Loss: 0.0003938 
[2301/3000][1/13] Loss: 0.0004376 
[2401/3000][1/13] Loss: 0.0005553 
[2501/3000][1/13] Loss: 0.0007593 
[2601/3000][1/13] Loss: 0.000

### Test model performance

In [299]:
# repeat for different number of layers in the neural network
for layer_number in range(start_layer, end_layer):
    # repeat for different sigma values for the kernel
    for hyper_number in range(start_hyper, end_hyper):
        # initializing variables for performance evaluation
        anal_index = []        
        anal_real_list = []
        anal_pred_list = []

        peak_position_real_list = []
        peak_position_pred_list = []
        peak_intensity_real_list = []
        peak_intensity_pred_list = []

        rmse_median = []
        rmse_data = []
        rsquare_median = []
        mae_median = []
        drmse_median = []
        
        # repeat for three times to check sensitivity to data splitting
        for random_number in range(3):
            # split data with different random seed
            X_train, X_test_val, Y_train, Y_test_val = train_test_split(X_data, Y_data, test_size=split_size, random_state=random_number) 
            X_test, X_val, Y_test, Y_val = train_test_split(X_test_val, Y_test_val, test_size=split_val_size, random_state=random_number)
            Dataset_train = torch.utils.data.TensorDataset(X_train, Y_train)
            loader_train = torch.utils.data.DataLoader(Dataset_train, batch_size=50, shuffle=True)
            Dataset_val = torch.utils.data.TensorDataset(X_val, Y_val)
            loader_val = torch.utils.data.DataLoader(Dataset_val, shuffle=False)

            # bulid model
            netG = KernelNet(
                n_neurons=n_neurons[layer_number - 1],
                wavelength_points=wavelength_points,
                kernel_grids=kernel_grids,
                kernel_func=RBFKernel(
                    sigmas_squared=kernel_hypers[hyper_number - 1][1],
                    hight=kernel_hypers[hyper_number - 1][0]
                ),
            ).to(device)
    
            #########
            ## Load trained model
            #########              
            trained_netG_path = '{}/generator_time{:03d}_layer{:03d}_hyper{:03d}.pth'.format(model_path, random_number, layer_number, hyper_number)
            netG.load_state_dict(torch.load(trained_netG_path))     

            ############
            ### Analysis
            ############
            pred_spectrum = netG(X_test.to(device))
            anal_real = Y_test[:,1:].to('cpu').detach().numpy().copy()
            anal_pred = pred_spectrum[:].to('cpu').detach().numpy().copy()
            anal_index.extend(Y_test[:,0].to('cpu').detach().numpy().copy())
            anal_real_list.extend(anal_real)
            anal_pred_list.extend(anal_pred)

            rmse_list = []
            rsquare_list = []
            mae_list = []
            drmse_list = []
        
            for i in range(anal_real.shape[0]):
                # RMSE
                rmse = np.sqrt(mean_squared_error(anal_real[i], anal_pred[i]))
                rmse_list.append(rmse)
                rmse_data.append(rmse)
                # R square
                rsquare = r2_score(anal_real[i], anal_pred[i])
                rsquare_list.append(rsquare)
                # MAE
                mae = mean_absolute_error(anal_real[i], anal_pred[i])
                mae_list.append(mae)
            # RMSE derivative
            real_d = anal_real[:,:-1] - anal_real[:,1:]
            pred_d = anal_pred[:,:-1] - anal_pred[:,1:]
            drmse = np.sqrt(np.sum(((real_d - pred_d) ** 2), axis=1) / anal_real.shape[0])
            drmse_list.append(drmse)
            # calculate median
            rmse_median.append(np.median(rmse_list))
            rsquare_median.append(np.median(rsquare_list))
            mae_median.append(np.median(mae_list))
            drmse_median.append(np.median(drmse_list))

        # all test results
        anal_index = np.array(anal_index, dtype='int32')
        anal_real_list = np.array(anal_real_list, dtype='float32')
        anal_pred_list = np.array(anal_pred_list, dtype='float32')


        print("#####################")
        print("##     output results  ##")
        print("#####################")
        print(netG)

        # output profiles
        device_cpu = torch.device('cpu')
        exp = pd.DataFrame(anal_real_list)
        exp.insert(loc = 0, column='smiles', value= smiles_list[anal_index])
        if case == 1:
            exp.to_csv('{}/exp_profiles_dataset1.csv'.format(out_result))
        elif case == 2:
            exp.to_csv('{}/exp_profiles_dataset2.csv'.format(out_result))
        pred = pd.DataFrame(anal_pred_list)
        pred.insert(loc = 0, column='smiles', value= smiles_list[anal_index])
        if case == 1:
            exp.to_csv('{}/pred_profiles_dataset1.csv'.format(out_result))
        elif case == 2:
            exp.to_csv('{}/pred_profiles_dataset2.csv'.format(out_result))
            
        # RMSE        
        rmse_df = pd.DataFrame([rmse_data])
        rmse_df = rmse_df.transpose()
        rmse_df.columns = ["rmse"]
        rmse_sort = rmse_df.sort_values('rmse')
        rmse_sort_index = rmse_sort.index.values
        print("Median of RMSE:", np.mean(rmse_median), "std:", np.std(rmse_median))
        # R2
        print("Median of R square:", np.mean(rsquare_median), "std:", np.std(rsquare_median))
        # MAE
        print("Median of MAE:", np.mean(mae_median), "std:",np.std(mae_median))            
        # dRMSE
        print("Median of RMSE_derivative", np.mean(drmse_median), "std:",np.std(drmse_median))
        

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

#####################
##     output results  ##
#####################
GNet(
  (fc_layers): SequentialLinear(
    (layer_0): LinearLayer(
      (linear): Linear(in_features=1024, out_features=896, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
      (normalizer): BatchNorm1d(896, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (layer_1): LinearLayer(
      (linear): Linear(in_features=896, out_features=640, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
      (normalizer): BatchNorm1d(640, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (layer_2): LinearLayer(
      (linear): Linear(in_features=640, out_features=512, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
      (normalizer): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activati